## Fleet Assignment + Revenue Management

### The weighted sum method is used for multiobjective optimization.

\begin{equation}
\text{Maximize} \;\; 

\alpha \cdot Revenue - \beta \cdot Cost, \;\; \text{where} \;\; \alpha + \beta = 1

\tag{1}
\end{equation}

We normalize the weight to make it easier to compare the ratio between them.

### Decision Variables

* $x_{i,j}$: binary indicator whether flight leg $i$ is assigned to the aircraft type $j$

* $soldE_i$: economy seats sold on flight leg $i$
* $soldB_i$: business seats sold on flight leg $i$

### Sets and Indices
* $F$ : set of flight legs (e.g., IST-YVR, YVR-IST, etc.), indexed by $i$.
* $G$ : set of aircraft types (e.g., B777-300ER, B787-9), indexed by $j$.
* $P$ : set of paired flights (outbound, return), indexed by $(i, i')$ (e.g., (TK75, TK76)).

* $E$ : set of Eco seat classes
* $B$ : set of Business seat classes


### Parameters (Inputs)
* $c_{i,j}$: operating cost of assigning aircraft $j$ to flight leg $i$
* $N_j$: number of available aircraft of fleet type

* $p_E$: ticket price for economy seat classes
* $p_B$: ticket price for business seat classes

* $capE_j$: economy seat capacity of aircraft type $j$
* $capB_j$: business seat capacity of aircraft type $j$

* $demE_i$: economy seat demand on flight $i$
* $demB_i$: business seat demand on flight $i$

### Basic Objective Function

\begin{equation}
\text{Maximize}\;\; 

\alpha \left( \sum_{i \in F}{p_E \cdot soldE_i + p_B \cdot soldB_i} \right)
 - \beta \sum_{i \in F}\sum_{j \in G}{c_{i,j} \cdot x_{i,j}}

\tag{2}
\end{equation}

#### Cost Breakdown

\begin{equation}
c_{i,j} = ...
\end{equation}